# Overview
Still To Do:
* **Complete**

The goal of this notebook is to handle missing state and sector electricity sales data and calculate kWh per capita consumption. For a model_ready dataset

Electric Sales - Sourced from EIA and kWh actually consumed
Per capita - Sourced from EIA and based on customer electricity accounts 

Notebooks sources:
* Electric Sales Ingestion - https://colab.research.google.com/drive/1MtcvMAsxUsEa0wHABkpf-sWS9h8XTJw4#scrollTo=kg5tIuCH1h97
* Electric Account Ingestion - https://colab.research.google.com/drive/1KTLQhMf1NVf6Y74d9tVHEQWEm8LPEqLl#scrollTo=JQ7wD-d6mm4N
* US Census Ingestion - https://colab.research.google.com/drive/1PYAjl4APZBQ7RTp_ERvW41d4n6tWkYIZ?usp=sharing
* Predict Electric Accounts with US Census State Population - https://drive.google.com/file/d/1VtRqJ4vwEh6EGD-DmEMPlROPdpKyXnCw/view?usp=sharing

# Terms 

**Source https://www.eia.gov/tools/glossary/index.php** <br>
Sectors include - Residential (RES), Commericial (COM), Transporation (TRA), Industrial (IND), and Other (OTH)

* RES -  An energy-consuming sector that consists of living quarters for private households. Common uses of energy associated with this sector include space heating, water heating, air conditioning, lighting, refrigeration, cooking, and running a variety of other appliances. The residential sector excludes institutional living quarters. Note: Various EIA programs differ in sectoral coverage.

* COM -  An energy-consuming sector that consists of service-providing facilities and equipment of businesses; Federal, State, and local governments; and other private and public organizations, such as religious, social, or fraternal groups. The commercial sector includes institutional living quarters. It also includes sewage treatment facilities. Common uses of energy associated with this sector include space heating, water heating, air conditioning, lighting, refrigeration, cooking, and running a wide variety of other equipment. Note: This sector includes generators that produce electricity and/or useful thermal output primarily to support the activities of the above-mentioned commercial establishments.

* TRA - An energy-consuming sector that consists of all vehicles whose primary purpose is transporting people and/or goods from one physical location to another. Included are automobiles; trucks; buses; motorcycles; trains, subways, and other rail vehicles; aircraft; and ships, barges, and other waterborne vehicles. Vehicles whose primary purpose is not transportation (e.g., construction cranes and bulldozers, farming vehicles, and warehouse tractors and forklifts) are classified in the sector of their primary use. Note: Various EIA programs differ in sectoral coverage.

* IND -  An energy-consuming sector that consists of all facilities and equipment used for producing, processing, or assembling goods. The industrial sector encompasses the following types of activity manufacturing (NAICS codes 31-33); agriculture, forestry, fishing and hunting (NAICS code 11); mining, including oil and gas extraction (NAICS code 21); and construction (NAICS code 23). Overall energy use in this sector is largely for process heat and cooling and powering machinery, with lesser amounts used for facility heating, air conditioning, and lighting. Fossil fuels are also used as raw material inputs to manufactured products. Note: This sector includes generators that produce electricity and/or useful thermal output primarily to support the above-mentioned industrial activities. Various EIA programs differ in sectoral coverage.

* Other - The "Other" category is defined as representing electricity consumers not elsewhere classified. This category includes public street and highway lighting service, public authority service to public authorities, railroad and railway service, and interdepartmental services.

# Decisions Made

* TRA sector - Some states had below 30 electric accounts for some states, which taken literally would mean below 30 accounts for vehicles whose primary purpose is transporting people or goods. This seems really low for a state. Therefore TRA is dropped
* Oth sector - Is not used so this is removed
* ALL sector - Is only used for aggregation validation purposes check Ingestion either notebook for more detail
* Populated 2001 - 2007 electric account data by using US census state population found here - https://drive.google.com/file/d/1VtRqJ4vwEh6EGD-DmEMPlROPdpKyXnCw/view?usp=sharing
* Manually filled in (AK 2016 RES) 

In [ ]:

from google.colab import drive

drive.mount('./drive/')

%cd drive/Shareddrives/Data606_Energy/


Drive already mounted at ./drive/; to attempt to forcibly remount, call drive.mount("./drive/", force_remount=True).
/content/drive/Shareddrives/Data606_Energy


In [ ]:
import sys
import random
import pandas as pd
import numpy as np
import glob
import json
from tqdm.autonotebook import tqdm
sys.path.append("helpers/")

from energygrid import EGRID

from helper_functions import combine_like_files,clean_eia_df
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
import sys


import plotly.express as px
from plotly.subplots import make_subplots

pd.set_option('display.float_format', lambda x: '%.5f' % x)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [ ]:
sys.path.append("helpers/")
energy_grid = EGRID()
energy_grid.get_states()

# Retail Electricty Sales in kWh

## Definition - Retail sales (electric):  Sales made directly to the customer that consumes the energy product.

Source - https://www.eia.gov/opendata/qb.php?category=38&sdid=ELEC.SALES.AL-ALL.M

# Read Files

## Per Capita

In [ ]:
#Read Electric Account Files
df_all_electric_accounts = pd.read_json("data/percapita/electric_accounts/json/combine/combine_electric_accounts.json")

df_all_electric_accounts_formatted = clean_eia_df(df_all_electric_accounts,col="electric_accounts")[["state","date","year","sector","electric_accounts"]]

df_all_electric_accounts_formatted.sample(5)

,state,date,year,sector,electric_accounts
19529,MN,2010-07-01,2010,ALL,2590424.00000
31708,IN,2014-05-01,2014,COM,348361.00000
33835,NC,2013-09-01,2013,COM,656440.00000
25737,GA,2009-05-01,2009,TRA,1.00000
12813,NV,2013-04-01,2013,IND,3862.00000


In [ ]:
#Read Census

df_census_data = (pd.read_csv('/content/drive/Shareddrives/Data606_Energy/data/percapita/us_census/us_census_2000_2020.csv')
                .rename(columns = {"abbrv":"state"})
                )

df_census_data.sample(5)

,state,year,state_population
706,UT,2013,2898773.00000
548,RI,2010,1053994.00000
116,IA,2002,2934234.00000
1029,GA,2020,10710017.00000
915,WI,2017,5793147.00000


## Electric Accounts

In [ ]:
# Read Electric Accounts
df_estimates_us_census_electricacc = (pd.read_csv('data/fill_ready/electric_accounts_fillin_values.csv')
                                        .drop(columns="Unnamed: 0")
                                        .melt(id_vars="year",var_name="sector_state",value_name="electric_accounts")
                                    )

df_estimates_us_census_electricacc[["sector","state"]] = df_estimates_us_census_electricacc["sector_state"].str.split("_",expand=True)

df_estimates_us_census_electricacc_set_index = df_estimates_us_census_electricacc.set_index(["state","year","sector"])

df_estimates_us_census_electricacc_set_index.sample(5)

sector_state  electric_accounts
state year sector                                
MT    2015 COM          COM_MT       104257.02753
      2011 IND          IND_MT         6397.37488
ND    2014 COM          COM_ND        69065.25873
NE    2002 COM          COM_NE       142858.86415
KS    2012 RES          RES_KS      1215479.86107

#Data Preperation Pipeline

## Pipeline Functions



In [ ]:
def plots():

    for sector in unique_sectors:
        
        query = df_electric_population_trim.query(f'(sector == "{sector}") & year > 2007 & (state=="TX" | state=="HI" | state=="CA")').sort_values(["year","sector"],ascending=True)

        colorsIdx = {"TX": "green", "CA": "blue","HI":"red"}

        color_cols = df_electric_population_trim.reset_index()['state'].map(colorsIdx)

        query_mean = (df_electric_population_trim.query(f'(sector == "{sector}") & year > 2007').sort_values(["year","sector"],ascending=True)
                    .groupby(["year"])).mean()


        vis = (px.scatter(query,x="year",y="kWh_percapita_electric_accounts",title=f"{sector} - kWh Per Capita Electricity Accounts",color="state"))
        
        vis.update_yaxes(matches=None)

        vis.add_trace(go.Scatter(x=query_mean.index, y=query_mean["kWh_percapita_electric_accounts"],
                            mode='lines',
                            name='USA Average'))


        fn = f"{sector}_kWh_percapita.html"
        vis.write_html(f"data/electricity_retail_sales/visuals/{fn}")

        vis.show()

## Per Capita Calculation

Sectors include - Residential (RES), Commericial (COM),and Industrial (IND)


$$ \text{kWh_percapita} = \frac{\text{kWh}}{\text{electric_accounts}}$$<br>$$ \text{kWh_percapita} = \frac{\text{kWh}}{\text{US_census_population}}$$

In [ ]:
  
def calc_percapita(df,col_1=None,col_2=None,new_col=None):

        #Calculate kWh per capita for electric accounts
        df[new_col] = (df[col_1] / df[col_2])

        return df


def rescale_MWh(df=None,col_name=None,new_col=None):

    df["kWh"] = df[col_name] * 1000000

    return df


def merge_datasets(df1,df2,features=None,join_type="left"):

    #Merge datasets to get electricity accounts
    df_electric_population = df1.merge(df2,on=features,how=join_type)

    return df_electric_population

def fill_inf(df,value):
   
   return df.replace(np.inf,value)

def fill_na(df,value):

    return df.fillna(value)


def trim_timeseries(df,year=None):

    return df.query(f'year >= {year}')




## Pipeline Applied

In [ ]:
#Read file
df_total_electric_retail_sales = pd.read_json("data/electricity_retail_sales/combine/combine_retail_sales.json")

df_electric_sales_formatted = (df_total_electric_retail_sales
                               .pipe(clean_eia_df,col="kWh_million")
                                .pipe(rescale_MWh,col_name="kWh_million",new_col = "kWh")
                                ) 

df_electric_sales_formatted.sample(5)


,series_id,name,f,description,source,updated,sector,country,state,kWh_million,date,year,kWh
54301,ELEC.SALES.ND-COM.M,Retail sales of electricity : North Dakota : c...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,ND,335.97642,2004-03-01,2004,335976420.00000
36218,ELEC.SALES.IL-RES.M,Retail sales of electricity : Illinois : resid...,M,Electricity sold to residential customers; Ele...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,RES,USA,IL,5056.74799,2008-07-01,2008,5056747990.00000
69235,ELEC.SALES.TX-OTH.M,Retail sales of electricity : Texas : other : ...,M,Electricity sold to the other category (rolled...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,OTH,USA,TX,0.00000,2015-04-01,2015,0.00000
29146,ELEC.SALES.NY-ALL.M,Retail sales of electricity : New York : all s...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,NY,13885.71011,2021-07-01,2021,13885710110.00000
30625,ELEC.SALES.SC-ALL.M,Retail sales of electricity : South Carolina :...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,SC,5618.79838,2001-03-01,2001,5618798380.00000


In [ ]:
#Filter date and removes OTH, TRA, and ALL from dataframe
#Calulates Per Capita Electricity Sales
#Fills in na and inf with value

df_merged_percapita = (df_electric_sales_formatted
                       .query('year <= 2021')
                       .query('sector != "OTH" & sector != "TRA" & sector != "ALL"')
                        #Merge
                       .pipe(merge_datasets,df_all_electric_accounts_formatted,features=["state","date","year","sector"],join_type="left")
                       .pipe(merge_datasets,df_census_data,features=["state","year"],join_type="left") 
                       #Calaculate
                       .pipe(calc_percapita,col_1="kWh",col_2="electric_accounts",new_col="percapita_electricity_sales_electric_accounts_kWh")
                       .pipe(calc_percapita,col_1="kWh",col_2="state_population",new_col="percapita_electricity_sales_state_population_kWh")
                       #Fill
                       .pipe(fill_inf,0)
                       .pipe(fill_na,0)
                        )

df_merged_percapita.sample(5)

,series_id,name,f,description,source,updated,sector,country,state,kWh_million,date,year,kWh,electric_accounts,state_population,percapita_electricity_sales_electric_accounts_kWh,percapita_electricity_sales_state_population_kWh
25149,ELEC.SALES.WY-RES.M,Retail sales of electricity : Wyoming : reside...,M,Electricity sold to residential customers; Ele...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,RES,USA,WY,154.59744,2004-09-01,2004,154597440.00000,0.00000,509106.00000,0.00000,303.66454
6531,ELEC.SALES.MO-IND.M,Retail sales of electricity : Missouri : indus...,M,Electricity sold to industrial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,IND,USA,MO,1493.63848,2012-06-01,2012,1493638480.00000,8908.00000,6026027.00000,167673.83026,247.86455
18966,ELEC.SALES.NH-RES.M,Retail sales of electricity : New Hampshire : ...,M,Electricity sold to residential customers; Ele...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,RES,USA,NH,307.79332,2005-05-01,2005,307793320.00000,0.00000,1298492.00000,0.00000,237.03906
34478,ELEC.SALES.OK-COM.M,Retail sales of electricity : Oklahoma : comme...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,OK,1895.23153,2009-06-01,2009,1895231530.00000,267213.00000,3717572.00000,7092.58730,509.80358
31642,ELEC.SALES.MT-COM.M,Retail sales of electricity : Montana : commer...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,MT,376.77629,2019-05-01,2019,376776290.00000,109436.00000,1070123.00000,3442.89164,352.08690


In [ ]:
#There are some year and state combinations that have zero electric accounts
#Those will be filled in later

(df_merged_percapita[["electric_accounts","year","sector","state","kWh"]]
 .groupby(["year","state"])
 .sum()
 .query('electric_accounts < 1')
)

electric_accounts                kWh
year state                                      
2001 AK               0.00000   5259902010.00000
     AL               0.00000  78619430020.00000
     AR               0.00000  40991709000.00000
     AZ               0.00000  59622137970.00000
     CA               0.00000 236168225000.00000
...                       ...                ...
2007 WA               0.00000  85740413980.00000
     WI               0.00000  71301300020.00000
     WV               0.00000  34179382000.00000
     WY               0.00000  15535551990.00000
2016 AK               0.00000   4116810010.00000

[358 rows x 2 columns]

In [ ]:
(px.line(df_merged_percapita
 .query('state == "DC"').sort_values("date")
 [["state","date","year","electric_accounts","state_population","kWh"
 ,"percapita_electricity_sales_electric_accounts_kWh"
 ,"percapita_electricity_sales_electric_accounts_kWh","sector"]]
 ,x="year",y="electric_accounts",color="sector"))

In [ ]:
df_merged_percapita.query('state == "DC" & sector == "RES"').sort_values("date")[["date","year","electric_accounts","state_population","kWh"
 ,"percapita_electricity_sales_electric_accounts_kWh"
 ,"percapita_electricity_sales_electric_accounts_kWh","sector"]].set_index("date").sample(10)

,year,electric_accounts,state_population,kWh,percapita_electricity_sales_electric_accounts_kWh,percapita_electricity_sales_electric_accounts_kWh,sector
date,,,,,,,
2011-08-01,2011,227805.00000,620290.00000,239881660.00000,1053.01315,1053.01315,RES
2008-02-01,2008,215853.00000,580236.00000,161787950.00000,749.52838,749.52838,RES
2004-08-01,2004,0.00000,567754.00000,217837000.00000,0.00000,0.00000,RES
2019-03-01,2019,280372.00000,708253.00000,212467930.00000,757.80723,757.80723,RES
2006-11-01,2006,0.00000,570681.00000,135775800.00000,0.00000,0.00000,RES
2008-06-01,2008,216290.00000,580236.00000,174826650.00000,808.29742,808.29742,RES
2006-02-01,2006,0.00000,570681.00000,149832340.00000,0.00000,0.00000,RES
2013-04-01,2013,235000.00000,651559.00000,128554260.00000,547.03940,547.03940,RES
2011-07-01,2011,228952.00000,620290.00000,271544300.00000,1186.03157,1186.03157,RES


In [ ]:
#Sectors in dataset after pipeline
df_merged_percapita.sector.unique().tolist()

['IND', 'RES', 'COM']

In [ ]:
#Graphical representation for spotting zero percapita values
#Note (AK RES 2016) and (AK RES 2016)

for sector in  df_merged_percapita.sector.unique():

    df_sector = df_merged_percapita.query(f'sector == "{sector}"').groupby(["year","state"]).sum().reset_index().sort_values(["year"])

    vis = px.line(df_sector,x="year",y="percapita_electricity_sales_electric_accounts_kWh",color="state",title=sector)

    vis.show()

In [ ]:
#Calculate mean percapita and locate which indexes need electric accounts filled in
df_electricacc_sector = (df_merged_percapita[["state","year","date","sector",
                                              "electric_accounts","kWh",
                                              "percapita_electricity_sales_electric_accounts_kWh",]]
                         
                        .groupby(["state","year","sector"])
                        .mean()
                        .reset_index()
                        .rename(columns={"kWh":"mean_kWh"})
                        )
 
df_statepop_sector = (df_merged_percapita[["state","year","sector",
                                           "percapita_electricity_sales_state_population_kWh"]]
                      
                        .groupby(["state","year","sector"])
                        .mean()
                        .reset_index()
                        .rename(columns={"kWh":"mean_kWh"})
                        )

# Fill in 2001-2008 and other zero values with mean electric accounts

In [ ]:
#Indexes where values will need to be filled in that are greater than 2008

df_to_fill = df_electricacc_sector.query('year > 2008 & percapita_electricity_sales_electric_accounts_kWh == 0')

df_to_fill

,state,year,sector,electric_accounts,mean_kWh,percapita_electricity_sales_electric_accounts_kWh
40,AK,2014,IND,0.00000,113306668.33333,0.00000
43,AK,2015,IND,0.00000,112627417.50000,0.00000
45,AK,2016,COM,0.00000,227624918.33333,0.00000
46,AK,2016,IND,0.00000,115442582.50000,0.00000
47,AK,2016,RES,0.00000,0.00000,0.00000
49,AK,2017,IND,0.00000,118380000.83333,0.00000
52,AK,2018,IND,0.00000,112625249.16667,0.00000
55,AK,2019,IND,0.00000,104312584.16667,0.00000
58,AK,2020,IND,0.00000,107160332.50000,0.00000
247,AZ,2020,IND,0.00000,1149471382.50000,0.00000


In [ ]:
def fill_values(df_fill,df_est,s_index=None):

    """
    df_fill: dataframe to be filled in
    df_est: dataframe to get estimate values from
    s_index: features to set the index to
    """

    df_fill_set_index = df_fill.set_index(s_index)
    
    df_est_set_index = df_est.set_index(s_index)

    #Create list of indexes that have 0 electricity accounts to interate over
    indexes = list(df_fill_set_index.query('percapita_electricity_sales_electric_accounts_kWh == 0').index)

    for index in indexes:

        df_fill_set_index.loc[index]["electric_accounts"] = df_est_set_index.loc[index]["mean_electric_accounts"]

        est_electric_accounts = df_fill_set_index.loc[index]["electric_accounts"]

        mean_kWh = df_fill_set_index.loc[index]["mean_kWh"]

        #calulate percapita
        df_fill_set_index.loc[index]["percapita_electricity_sales_electric_accounts_kWh"] =  mean_kWh / est_electric_accounts


    return df_fill_set_index,indexes





In [ ]:
#Estimate Dataset was created by this notebook - https://colab.research.google.com/drive/1VtRqJ4vwEh6EGD-DmEMPlROPdpKyXnCw#scrollTo=iSO4Lkkvv0_b

df_est_census_to_electric_accounts = (pd.read_csv("data/fill_ready/electric_accounts_fillin_values.csv")
                                    .drop(columns=["Unnamed: 0"])
                                    .melt(id_vars="year",var_name="sector_state",value_name="mean_electric_accounts")
                                    )

df_est_census_to_electric_accounts[["sector","state"]] = df_est_census_to_electric_accounts["sector_state"].str.split("_",expand=True)

df_est_census_to_electric_accounts.head()

,year,sector_state,mean_electric_accounts,sector,state
0,2000,RES_AK,252927.47063,RES,AK
1,2001,RES_AK,254880.10601,RES,AK
2,2002,RES_AK,256791.68507,RES,AK
3,2003,RES_AK,258739.66016,RES,AK
4,2004,RES_AK,260619.08892,RES,AK


In [ ]:
#Fill in values

df_filled,fill_indexes = (fill_values(df_electricacc_sector,df_est_census_to_electric_accounts,s_index=["state","sector","year"]))

In [ ]:
#Validation on expected amount of filled in indexes

count_point_less_2008 = len(df_electricacc_sector.query('year < 2008'))
count_zero_electric_accounts_values = len(df_to_fill)

assert count_point_less_2008 + count_zero_electric_accounts_values == len(fill_indexes)

## **Manual Filling of (AK,RES,2016)**

In [ ]:
# Still AK RES 2015 left to fill
df_filled.query('percapita_electricity_sales_electric_accounts_kWh == 0')

,,,electric_accounts,mean_kWh,percapita_electricity_sales_electric_accounts_kWh
state,sector,year,,,
AK,RES,2016,283846.61841,0.00000,0.00000


In [ ]:
# Populate AK RES 2016 mean_kWh with mean of AK RES 2015 and 2017

manual_input = ("AK","RES",2016)

assert fill_indexes.index(manual_input)

mean_kWh_manual_fill = (df_filled.query('state == "AK"')
                        .query('sector == "RES"')
                        .query('year == 2015 | year == 2017')
                        ["mean_kWh"]
                        .mean()
                        )

df_filled.loc[manual_input]["mean_kWh"] = mean_kWh_manual_fill

mean_kWh = df_filled.loc[manual_input]["mean_kWh"]

electric_acounts = df_filled.loc[manual_input]["electric_accounts"]

df_filled.loc[manual_input]["percapita_electricity_sales_electric_accounts_kWh"] = mean_kWh / electric_acounts

In [ ]:
# All 0 values have been taken care of

df_filled.query('year >= 2001 & percapita_electricity_sales_electric_accounts_kWh == 0')

,,,electric_accounts,mean_kWh,percapita_electricity_sales_electric_accounts_kWh
state,sector,year,,,


In [ ]:
df_filled.to_csv("data/eda_ready/electricity_sales_2001_2020.csv")

In [ ]:

vis = (px.line(df_filled.reset_index().sort_values("year")
,x="year",y="percapita_electricity_sales_electric_accounts_kWh",color="state"
,color_discrete_sequence=px.colors.qualitative.G10,facet_col="sector"))

vis.update_yaxes(matches=None)

vis.show()


In [ ]:
df_filled.query('state=="DC" & sector == "IND"')

electric_accounts  ...  percapita_electricity_sales_electric_accounts_kWh
state sector year                     ...                                                   
DC    IND    2001            1.00000  ...                                     23409499.16667
             2002            1.00000  ...                                     23533083.33333
             2003            1.00000  ...                                     22270667.50000
             2004            1.00000  ...                                     23504165.83333
             2005            1.00000  ...                                     21370582.50000
             2006            1.00000  ...                                     19986834.16667
             2007            1.00000  ...                                     24723916.66667
             2008            1.00000  ...                                     21420499.16667
             2009            1.00000  ...                                     19502918.33333
             2010            1.00000  ...                                     19133083.33333
             2011            1.00000  ...                                     18020333.33333
             2012            1.00000  ...                                     18198001.66667
             2013            1.00000  ...                                     18948750.83333
             2014            1.00000  ...                                     20179082.50000
             2015            1.00000  ...                                     19852666.66667
             2016            1.00000  ...                                     16037832.50000
             2017            1.00000  ...                                     15015250.83333
             2018            1.00000  ...                                     16055167.50000
             2019            1.00000  ...                                     14969335.00000
             2020            1.00000  ...                                     16780841.66667
             2021            1.00000  ...                                     18768580.00000

[21 rows x 3 columns]

In [ ]:
# Visual Check

for sector in df_filled.reset_index().sector.unique():

    query = df_filled.reset_index().query(f'sector == "{sector}"')

    vis = px.line(query,x="year",y="percapita_electricity_sales_electric_accounts_kWh",color="state",title=sector)
    
    vis.show()

In [ ]:
def pivot_df(df,index=None,columns=None,level=None,axis=None,suffix=None):

    return df.pivot(index=["state","year"],columns="sector").droplevel(level,axis=axis).add_suffix(suffix)



def percent_normalize(df):

    cols = df.columns

    df["total"] = df.sum(axis=1)

    for col in cols:

        df[col] = round(df[col] / df["total"],3)

    return df





In [ ]:
df_electricacc_norm = (df_filled.reset_index().drop(columns=["electric_accounts","mean_kWh"])
                        .pipe(pivot_df,index=["state","year"],level=0,axis=1,columns="sector",suffix="_percapita_mean_kWh_electricacc")
                        .pipe(percent_normalize)
                        .drop(columns="total")
                        )


df_statepop_norm = (df_statepop_sector
                        .pipe(pivot_df,index=["state","year"],level=0,axis=1,columns="sector",suffix="_percapita_mean_kWh_statepop")
                        .pipe(percent_normalize)
                        .drop(columns="total")
                    )

In [ ]:
df_statepop_norm['COM_percapita_mean_kWh_statepop'].isna().sum()

51

In [ ]:
df_merge_ready = df_electricacc_norm.merge(df_statepop_norm,left_index=True,right_index=True,how="left")

df_merge_ready.sample(5)

,sector,COM_percapita_mean_kWh_electricacc,IND_percapita_mean_kWh_electricacc,RES_percapita_mean_kWh_electricacc,COM_percapita_mean_kWh_statepop,IND_percapita_mean_kWh_statepop,RES_percapita_mean_kWh_statepop
state,year,,,,,,
KS,2018,0.12300,0.85700,0.02000,0.38500,0.27800,0.33700
MO,2013,0.04600,0.94700,0.00700,0.36600,0.21000,0.42400
WY,2010,0.06000,0.93200,0.00900,0.25200,0.58800,0.15900
NE,2001,0.10100,0.87600,0.02300,0.31200,0.31600,0.37200
WI,2016,0.01500,0.98300,0.00200,0.34200,0.34500,0.31300


# Save Dataframe to Merge_Ready

In [ ]:
#df_merge_ready.to_csv("data/fill_ready/annual_electricity_sales.csv")

remove_ind_columns = [col for col in df_merge_ready if not col.startswith("IND")]

df_merge_ready[remove_ind_columns].to_csv("data/merge_ready/annual_2001_2021_electricity_sales_RES_COM.csv")

In [ ]:
df_corr = pd.read_csv("data/fill_ready/annual_electricity_sales.csv")

In [ ]:
df_corr.sort_values(["state","year"]).corr()